pdfminer3k  
pdfminer.six
pypdf  
pypdfocr


In [1]:
phone_numbers = ["./data/phone_numbers/070219_KV_BR_Telefonverzei.pdf",
                "./data/phone_numbers/52_pdf_wegweiser_schlaganfall.pdf",
                 "./data/phone_numbers/FL_SYB_BetriebsaerztlicherDienst_ID8414.pdf",
                 "./data/phone_numbers/Infoblatt_154.pdf"
                ]
scans = ["./data/scans/double_ocr.pdf",
         "./data/scans/single_ocr.pdf"]

### pdfminer3k

In [2]:
#!/usr/bin/env python3
#-*- coding:utf-8 -*-
# pip3 install pdfminer3k
  
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBoxHorizontal
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter, PDFTextExtractionNotAllowed
from pdfminer.pdfdevice import PDFDevice
 
def read_pdf(pdf_name, result_name):
    # 以二进制读模式打开
    fp = open(pdf_name, 'rb')
    # 用文件对象来创建一个pdf文档分析器
    parser = PDFParser(fp)
    # 创建一个pdf文档
    doc = PDFDocument()
    # 连接分析器 与文档对象
    parser.set_document(doc)
    doc.set_parser(parser)
    # 提供初始密码，如果没有密码 就创建一个空的字符串
    doc.initialize('')
    # 检测文档是否提供txt转换，不提供就抛出异常
    if not doc.is_extractable:
        raise PDFTextExtractionNotAllowed
    # 创建PDf 资源管理器 来管理共享资源
    rsrcmgr = PDFResourceManager()
    # 创建一个PDF设备对象
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    # 创建一个PDF解释器对象
    interpreter = PDFPageInterpreter(rsrcmgr, device)
     
    with open(result_name,"w",encoding="utf-8") as fd_out:
        # 循环遍历列表，每次处理一个page的内容
        for i,page in enumerate(doc.get_pages(),1):
            index = "===========《第{}页》===========".format(i)
            print(index)
            fd_out.write(index + "\n")
            interpreter.process_page(page)
            # 接受该页面的LTPage对象
            layout = device.get_result()
            for x in layout:
                if hasattr(x,"get_text"):
                    results = x.get_text()
                    print(results)
                    fd_out.write(results) 
                # 这里layout是一个LTPage对象 里面存放着 这个page解析出的各种对象 一般包括LTTextBox,
                # LTFigure, LTImage, LTTextBoxHorizontal 等等 想要获取文本就获得对象的text属性
                #if not isinstance(x, LTTextBoxHorizontal):
                #    continue
                #results = x.get_text()
                #print(results)
                #fd_out.write(results)   
 
                         
 

pdf_name = phone_numbers[3]
result = './test.txt'
read_pdf(pdf_name, result)

ImportError: cannot import name 'PDFDocument' from 'pdfminer.pdfparser' (C:\ProgramData\Anaconda3\lib\site-packages\pdfminer\pdfparser.py)

In [11]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import os 

def convert_pdf_to_txt(path, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(path, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()
    return text

s = convert_pdf_to_txt(phone_numebrs[0])
print(s)

ModuleNotFoundError: No module named 'pdfminer.pdfpage'

### pdfminer.six

In [4]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

def convert(case,fname, pages=None):
    if not pages: pagenums = set();
    else:         pagenums = set(pages);      
    manager = PDFResourceManager() 
    codec = 'utf-8'
    caching = True

    if case == 'text' :
        output = io.StringIO()
        converter = TextConverter(manager, output, codec=codec, laparams=LAParams())     
    if case == 'HTML' :
        output = io.BytesIO()
        converter = HTMLConverter(manager, output, codec=codec, laparams=LAParams())

    interpreter = PDFPageInterpreter(manager, converter)   
    infile = open(fname, 'rb')

    for page in PDFPage.get_pages(infile, pagenums,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    convertedPDF = output.getvalue()  

    infile.close(); converter.close(); output.close()
    return convertedPDF

#//////////// main ///////////////////////
filePDF  = phone_numbers[0]     # input
fileHTML = 'myHTML.html'   # output
fileTXT  = 'myTXT.txt'     # output

case = "text"

if case == 'HTML' :
    convertedPDF = convert('HTML', filePDF)
    fileConverted = open(fileHTML, "wb", encoding="utf-8")
if case == 'text' :
    convertedPDF = convert('text', filePDF)
    fileConverted = open(fileTXT, "w", encoding="utf-8")

fileConverted.write(convertedPDF)
fileConverted.close()
#print(convertedPDF) 

### pdftotree

pyocr

In [4]:
# import pdftotree
import subprocess
import pyocr
import importlib
import sys,os
import time
import requests

importlib.reload(sys)
time1 = time.time()



def pdf_to_html(pdf_path):
    """
    将pdf转HTML，成功与不成功的
    :param pdf_path: pdf的本地路径
    :return:如果该文件是扫描版，返回扫描版，如果转换失败返回False,如果成功返回转化成HTML的地址
    """
    try:
        pdf_file = pdf_path
        html_output = pdf_file.split(".")[0]+".html"
        html_output_path = html_output.split("/")[-1]
        cmd = 'pdftotree' + " " + "-o" + " " + html_output + " " + pdf_file
        b = run_command(cmd.split())
        for i in b:
            i = i.decode()
            if "cannot build tree structure" in i:
                raise Exception("该文件是PDF扫描版")
            if "tabula.errors.JavaNotFoundError" in i:
                raise Exception(
                    "tabula.errors.JavaNotFoundError: `java` command is not found from this Python process. "
                    "Please ensure Java is installed and PATH is set for `java`")
            info(f"{pdf_file}已经转换成HTML")
            html_output_path = html_output_path+html_output_path
            domain_url = "/Users/gongsi/PycharmProjects/sticker/formal/AIQT_DATA/citic_report/citic_report/down_pdf/"
            file_path = os.path.join(os.path.dirname(__file__),)
            print(">>>>file",file_path)
            if os.path.exists(domain_url+html_output_path):

                return html_output_path
            return False
    except Exception as e:
        print('出现错误了', e)
        if "PDF扫描版" in e:
            return "PDF扫描版"
        return False
    
def run_command(command):
    """
    执行cmd命令
    :param command: cmd字符串
    :return: 包含执行命令结果的一个迭代器
    """
    p = subprocess.Popen(command,
                         stdout=subprocess.PIPE,
                         stderr=subprocess.STDOUT)
    return iter(p.stdout.readline, b'')

pdf_to_html("./ITA_asg_1.pdf")

出现错误了 [WinError 2] The system cannot find the file specified


TypeError: argument of type 'FileNotFoundError' is not iterable

### PDF TO Text
For Debian, Ubuntu, and friends
```
sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
```
For Fedora, Red Hat, and friends
```
sudo yum install gcc-c++ pkgconfig poppler-cpp-devel python3-devel
```

In [2]:
import pdftotext

 # Load your PDF
with open("./data/phone_numbers/070219_KV_BR_Telefonverzei.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)

 # Save all text to a txt file.
with open('test.txt', 'w') as f:
    f.write("\n\n".join(pdf))

ModuleNotFoundError: No module named 'pdftotext'